In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mp_api.client import MPRester
import pymatgen.core.structure
import random
from pymatgen.analysis.chemenv.utils.coordination_geometry_utils import Plane
import numpy as np
from scipy.spatial import distance
import random

## Development

### Getting material structure

In [ ]:
from PIL import Image

def resize_image(image_path, new_width, new_height):
    """Resizes an image while maintaining aspect ratio.

    Args:
        image_path (str): The path to the image file.
        new_width (int): The desired new width of the image.
        new_height (int): The desired new height of the image.
    """

    try:
        img = Image.open(image_path)

        # Calculate aspect ratio and resize accordingly
        ratio = new_width / img.width  
        if ratio * img.height > new_height:  # Resize based on height if needed
            ratio = new_height / img.height
            new_width = int(ratio * img.width)

        resized_img = img.resize((new_width, int(ratio * img.height))) 
        resized_img.save(image_path)  # Overwrite the original file

    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
image_path = "C:\\Users\\91931\\~\\diss\\PLOT_GAUSSIAN_2.png"  # Replace with your image path
new_width = 256
new_height = 256
resize_image(image_path, new_width, new_height)


In [ ]:
with MPRester("jS4ST5fsFePAWwMurwaUh5FRcfXmgdA3") as mpr:
    list_of_available_fields = mpr.summary.available_fields
    print(list_of_available_fields)
    # docs = mpr.summary.search(fields=["material_id", "structure", "nsites", "volume", "energy_above_hull"])

In [ ]:
# print(docs[20].energy_above_hull)
docs_NONE = [x for x in docs if x.energy_above_hull == None]
print(len(docs_NONE))
docs_filtered=[]
for x in docs:
    if x.energy_above_hull == None:
        print(x)
    elif x.energy_above_hull < .1:
        docs_filtered.append(x)

len(docs_filtered)

### Converting to pymatgen structure

In [ ]:
mp_757220[0].volume

In [ ]:
# Assuming doc is your MPDataDoc<SummaryDoc> object
structure = mp_757220[0].structure  # Replace 'structure' with the actual attribute name
print(type(structure))
print(structure.lattice.a)
print("\n")
print(structure)
cc=VolumeScaling(mp_757220[0])
print(cc.structure)
# print(len(structure))
# print(structure[5])
# print(structure[5].coords)

In [ ]:
structure.lattice

### Making a pymatgen plane from three random points of the structure

In [ ]:

total_atoms = len(structure)

# Pick three random atom indices
threeRandomPointCoords = random.sample(range(total_atoms), 3)
print(threeRandomPointCoords)
points=[]
for x in threeRandomPointCoords:
    points.append(structure[x].coords)
print(points)

plane = Plane.from_3points(*points)


### Finding closest points to the plane

In [ ]:
def getPointsCoordsInStructure(structure):
    return list(map(lambda site: site.coords, structure[:]))


In [ ]:
coords_list = list(map(lambda site: site.coords, structure[:]))
coords_list=np.array(coords_list)

print(len(coords_list))

print(plane.distances_indices_sorted(coords_list, sign=False))
print(plane.distances(coords_list))

### Ranking the combinations of three points based on how many other points are in the plane they create

In [ ]:
# import numpy as np
# from itertools import combinations

# def plane_from_points(p1, p2, p3):
#     v1 = p3 - p1
#     v2 = p2 - p1
#     cp = np.cross(v1, v2)
#     a, b, c = cp
#     d = np.dot(cp, p3)
#     return a, b, c, d

# def point_plane_distance(a, b, c, d, points):
#     distances = np.abs(np.dot(points, np.array([a, b, c])) - d) / np.sqrt(a**2 + b**2 + c**2)
#     return distances

# def rank_planes(points):
#     points_array = np.array(points)
    
#     # Compute plane equations and inliers in one step
#     combinations_array = np.array(list(combinations(points_array, 3)))
#     cp_array = np.cross(combinations_array[:, 2] - combinations_array[:, 0], combinations_array[:, 1] - combinations_array[:, 0])
#     d_array = np.einsum('ij,ij->i', cp_array, combinations_array[:, 2])
#     a, b, c = cp_array.T
    
#     distances = point_plane_distance(a, b, c, d_array, points_array)
#     inliers_count = np.sum(distances < 0.01, axis=1)

#     # Combine results and sort
#     plane_rankings = sorted(zip(combinations_array, inliers_count), key=lambda x: x[1], reverse=True)

#     return plane_rankings


In [ ]:
# rankings=rank_planes(coords_list)
# print(rankings)

In [ ]:
average_loss = np.mean(loss_values)
average_mse = np.mean(mse_values)
average_rmse = np.mean(rmse_values)
average_mae = np.mean(mae_values)

# Print the averages
print("Average Loss:", average_loss)
print("Average MSE:", average_mse)
print("Average RMSE:", average_rmse)
print("Average MAE:", average_mae)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
def apply_gaussian_and_plot_fixed_dimensions(centers, mpid,plotCounter, amplitude=1.0, sigma=.2, target_resolution=(256, 256), plane_length=40, plane_width=40):
    # Calculate Aspect Ratio
    aspect_ratio = plane_width / plane_length
    
    # Calculate resolution based on aspect ratio
    resolution_x = int(np.sqrt(np.prod(target_resolution) / aspect_ratio))
    resolution_y = int(resolution_x * aspect_ratio)

    # Define the bounds of the plane
    min_x = -plane_length / 2
    max_x = plane_length / 2
    min_y = -plane_width / 2
    max_y = plane_width / 2

    # making the grid of points in the image
    x = np.linspace(min_x, max_x, resolution_x)
    y = np.linspace(min_y, max_y, resolution_y)
    x, y = np.meshgrid(x, y)

    total_gaussian_value = np.zeros_like(x)

    for center in centers:
        distance_squared = (x - center[0])**2 + (y - center[1])**2
        gaussian_value = amplitude * np.exp(-distance_squared / (2 * sigma**2))
        total_gaussian_value += gaussian_value

    # Plot
    plt.figure(figsize=(8, 8))
    plt.axis('off')
    plt.tight_layout()
    # plt.imshow(total_gaussian_value, extent=(min_x, max_x, min_y, max_y), origin='lower', cmap='gray')
    plt.savefig(f"ECDDATAEMERGENCY/PLOT_GAUSSIAN_{random.randint(0, 1000000)}_{mpid}.png", bbox_inches='tight', pad_inches=0)
    plt.axis('off')  # Turn off axis lines
    plt.close()

# Sample data
centers = [(0, 0), (5, 5), (-5, -5)]

# Example usage
apply_gaussian_and_plot_fixed_dimensions(centers, 0, 0, amplitude=1.0, sigma=.3, target_resolution=(256, 256))


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt

# def apply_gaussian_and_plot_fixed_dimensions(centers, amplitude=1.0, sigma=1.0, fixed_resolution=256):
#     min_x, max_x = -5, 5
#     min_y, max_y = -5, 5

#     # aspect_ratio = (max_y - min_y) / (max_x - min_x)  # Calculate Aspect Ratio
#     # resolution_x = int(np.sqrt(fixed_resolution / aspect_ratio))
#     # resolution_y = int(resolution_x * aspect_ratio)

#     # Making the grid of points in the image
#     x = np.linspace(min_x, max_x)
#     # print(x)
#     y = np.linspace(min_y, max_y)
#     # print(y)
#     x, y = np.meshgrid(x, y)

#     total_gaussian_value = np.zeros_like(x)

#     for center in centers:
#         distance_squared = (x - center[0])**2 + (y - center[1])**2
#         gaussian_value = amplitude * np.exp(-distance_squared / (2 * sigma**2))
#         total_gaussian_value += gaussian_value

#     plt.figure(figsize=(8, 8))
#     plt.title("2D Gaussian Glow (Multiple Points)")
#     plt.xlabel("X-axis")
#     plt.ylabel("Y-axis")
#     plt.imshow(total_gaussian_value, extent=(min_x, max_x, min_y, max_y), origin='lower', cmap='viridis')
#     plt.colorbar(label='Brightness')
#     plt.scatter([c[0] for c in centers], [c[1] for c in centers], color='red', marker='o', label='Centers')
#     plt.legend()
#     plt.show()

# # Example usage
# centers = [(-5, -5), (2, 3), (-1, -2)]


# apply_gaussian_and_plot_fixed_dimensions(centers, amplitude=1.0, sigma=0.5)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image



# import numpy as np
# import matplotlib.pyplot as plt

# def plot_points_with_gradient(width, height, points):
#     plane = np.zeros((height, width))

#     for point in points:
#         x, y = point
#         sigma = 30  # Adjust the standard deviation to control the spread of the glow
#         A = 1 / (2 * np.pi * sigma**2)

#         for i in range(height):
#             for j in range(width):
#                 distance_squared = (j - x)**2 + (i - y)**2
#                 falloff = A * np.exp(-distance_squared / (2 * sigma**2))
#                 plane[i, j] += falloff

#     # Normalize the values
#     plane /= np.max(plane)

#     # Create a gradient from white to black
#     gradient = np.linspace(1, 0, 256).reshape(1, -1)
#     colored_plane = np.multiply(gradient, plane[:, :, None])  # Add an extra dimension for the color

#     # Display the plot
#     plt.imshow(colored_plane, cmap='gray', origin='lower')
#     plt.show()

# # Example usage:
# width = 100
# height = 80
# points = [(20, 30), (50, 70), (80, 10), (30, 50)]

# plot_points_with_gradient(width, height, points)


# # Example usage:
# min_x = 0
# max_x = 10
# min_y = 0
# max_y = 8
# points = [(2, 3), (5, 7), (8, 1), (3, 5)]

# import numpy as np

# def calculate_brightness(points, query_point, dissipation_factor):
#     brightness = 0.0

#     for point in points:
#         x, y = point
#         distance_squared = (query_point[0] - x)**2 + (query_point[1] - y)**2
#         falloff = np.exp(-distance_squared / (2 * dissipation_factor**2))
#         brightness += falloff

#     return brightness

# # Example usage
# points = [(0, 0), (3, 4), (-2, -1)]
# query_point = (1, 2)
# dissipation_factor = 1.0

# brightness = calculate_brightness(points, query_point, dissipation_factor)
# print(f"Brightness at {query_point}: {brightness}")


# def create_glowing_planes(min_x, max_x, min_y, max_y, points):
#     target_size=(256, 256)
    
#     height = math.ceil(abs(min_y-max_y)) * resolution_factor
#     width = math.ceil(abs(min_x-max_x)) * resolution_factor
#     plane = np.zeros((height, width))
#     for point in points:
#         x, y = point
#         sigma = 20  # Adjust the standard deviation to control the spread of the glow
#         A = 1 / (2 * np.pi * sigma**2)
        
#         for i in range(height):
#             for j in range(width):
#                 distance_squared = (j/resolution_factor - x)**2 + (i/resolution_factor - y)**2
#                 falloff = np.exp(-distance_squared / (2 * sigma**2))
#                 plane[i, j] += A * falloff

#     # Normalize the values
#     plane /= np.max(plane)
#     # plane = (plane * 255).astype(np.uint8)
#     # # # plt.axis('off')
#     plt.xlim(min_x, max_x)
#     plt.ylim(min_y, max_y)
#     plt.tight_layout()
#     plt.imshow(plane, origin='lower')
#     # resized_img = Image.fromarray(plane_int).resize(target_size, Image.LANCZOS)
#     # resized_img.save(f'NormalisedData/Plane_{random.randint(1, 100)}.png')
    
#     # plt.savefig(f'NormalisedData/Plane_{random.randint(1, 100)}.png', bbox_inches='tight', pad_inches=0)
#     # plt.show()
    

# # Show the plot
# create_glowing_planes(min_x, max_x, min_y, max_y, points)

In [ ]:
trainingCombinations=[]

In [ ]:
import numpy as np
from itertools import combinations
import math
import random

random.seed(2024)
plotCounter=0
global plotCounter
def find_indices_strict(lst, value):
    return [index for index, element in enumerate(lst) if element == value]

def find_indices_conditional(lst, value):
    return [index for index, element in enumerate(lst) if abs(element - value) < .01]

def select_elements_by_indices(my_list, indices):
    return [my_list[index] for index in indices]

def find_dims_of_plane(points):
    x_coordinates, y_coordinates = zip(*points)

    # Determine the extent of the plane
    min_x = min(x_coordinates)
    max_x = max(x_coordinates)
    min_y = min(y_coordinates)
    max_y = max(y_coordinates)
    
    return([math.ceil(min_x)-2, math.ceil(max_x)+2, math.ceil(min_y)-2, math.ceil(max_y)+2])

def getPointsCoordsInStructure(structure):
    return list(map(lambda site: site.coords, structure[:]))

def VolumeScaling(ChemicalObject):
    reformedLattice = ChemicalObject.structure.lattice.scale(ChemicalObject.nsites)    
    ChemicalObject.structure.lattice = reformedLattice
    return ChemicalObject


# def shortestPairWiseDistance(points):
#     distances = cdist(points, points)
#     # Set the diagonal elements to a large value (e.g., infinity) to avoid considering the distance from a point to itself
#     np.fill_diagonal(distances, np.inf)
#     # Find the indices of the minimum distance
#     min_distance_indices = np.unravel_index(np.argmin(distances), distances.shape)
#     min_distance = distances[min_distance_indices]

def rank_planes(structure, mpid, supercell_dim=15):
    a = structure.lattice.a
    b = structure.lattice.b
    c = structure.lattice.c
    points=getPointsCoordsInStructure(structure)
    print(1)
    points_array = np.array(points)
    print(2)
    print(f"\tPoints in supercell to iterate through: {len(structure)}") 
    combinations_array = np.array(list(combinations(points_array, 3)))  # make all the possible combos of 3 given a list of points
    trainingCombinations.append(combinations_array)
    ## check lattice parameters  (one of the lattice parameters is a smaller number than the other)
    print(3)
    supercell_structure = structure.make_supercell([math.ceil(supercell_dim / a),
                                                    math.ceil(supercell_dim / b),
                                                    math.ceil(supercell_dim / c)])
    combo_no=0
    TEMP_coordsFullList=[]
    for combo in combinations_array[:-1]:
        print(f"Combination no. {combo_no} of {len(combinations_array)}")
        plane1 = Plane.from_3points(*combo)
        coords_list = getPointsCoordsInStructure(supercell_structure)
        distancesAndIndices = plane1.distances_indices_sorted(coords_list)
        indices_on_plane = [index for index, value in enumerate(map(lambda x: abs(x), distancesAndIndices[0])) if value < 0.1]
        print(f"\tNo. of points on this plane: {len(indices_on_plane)}")
        pointsOnPlane = select_elements_by_indices(coords_list, indices_on_plane)
        coordsOnPlaneForPoints = plane1.project_and_to2dim(pointsOnPlane, "mean")
        TEMP_coordsFullList.append(coordsOnPlaneForPoints)
        # print(coordsOnPlaneForPoints)
        apply_gaussian_and_plot_fixed_dimensions(coordsOnPlaneForPoints,plotCounter,mpid)
        combo_no += 1
    return TEMP_coordsFullList

def main(ListOfIDS):
    MATERIALNO = 1
    trainingCombinations=[]
    for x in ListOfIDS[:]:  # Iterate over a copy of the list
        try:
            ChemicalObject = mpr.summary.search(material_ids=[x], fields=["structure", "nsites", "volume"])[0]
        except Exception as e:
            print(f"An error occurred: {e}")
            continue

        if len(VolumeScaling(ChemicalObject).structure) < 2000:
            print(f"-----------------------  M A T E R I A L  N o.  {MATERIALNO}: {x}  -----------------------")
            pointsOnPlaneWithDimOfPlanes = rank_planes(VolumeScaling(ChemicalObject).structure, x)
            trainingCombinations.append(pointsOnPlaneWithDimOfPlanes)
            if len(trainingCombinations) % 10000:
                print(len(trainingCombinations))
            if len(trainingCombinations) >= 500000:
                break
        else:
            continue

        MATERIALNO+=1
        

        

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("C:\\Users\\91931\\~\\diss\\materialProjectCompleteWithFileNameReference - Copy.csv")

# Extract the 'material_id' column
material_ids = df['material_id']

# Now material_ids contains only the 'material_id' column data
print(material_ids)


In [ ]:
## TEST OF ALL
percentage=5
num_items_to_select = int(len(material_ids) * (percentage / 100))

# Select random items from the list
IDS = random.sample(list(material_ids), num_items_to_select)
# print(len(IDS))
# IDS=[]

main(IDS)

In [ ]:
print(len(trainingCombinations))

In [ ]:
del structure

In [ ]:
from itertools import chain
flattened_list = list(chain.from_iterable(trainingCombinations))
# print(flattened_list)

In [ ]:
len(flattened_list)

In [ ]:
import pickle
with open('training_combinations_3.pkl', 'wb') as f:
    pickle.dump(flattened_list, f)

In [ ]:

import cv2

# Load the image
image = cv2.imread("C:\Users\91931\~\diss\ECDDATAEMERGENCY\PLOT_GAUSSIAN_251754_0.png", cv2.IMREAD_GRAYSCALE)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# Threshold the image to create a binary image
_, thresholded = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)

# Find contours in the binary image
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw contours on the original image
image_with_contours = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), contours, -1, (0, 255, 0), 2)

# Count the number of detected contours (glows)
num_glows = len(contours)
print("Number of glows:", num_glows)

# Display the image with detected contours
cv2.imshow('Image with Contours', image_with_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import os
def detect_glows(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(image, (5, 5), 0)

    # Threshold the image to create a binary image
    _, thresholded = cv2.threshold(blurred, 100, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    image_with_contours = cv2.drawContours(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), contours, -1, (0, 255, 0), 2)

    # Count the number of detected contours (glows)
    num_glows = len(contours)
    return num_glows

# Example usage:
root_path="C:\\Users\\91931\\~\\diss\\ECDDATAEMERGENCY"
names = os.listdir(root_path)
paths=[os.path.join(root_path,name) for name in names]

i=int(0)
numberofsites=[]
print("starting loop")
for path in paths:
   if i<10:
       print(i)
   elif i/1000==0:
       print(i/1000)
   numberofsites.append(detect_glows(path)) 
   i+=1

In [ ]:
root_path="C:\\Users\\91931\\~\\diss\\ECDGENERATED DATA"
names = os.listdir(root_path)
paths=[os.path.join(root_path,name) for name in names]

i=int(0)
numberofsitesGEN=[]
print("starting loop")
for path in paths:
   if i<10:
       print(i)
   elif i/1000==0:
       print(i/1000)
   numberofsitesGEN.append(detect_glows(path)) 
   i+=1

In [ ]:
print(len(numberofsites))
print(len(numberofsitesGEN))

In [ ]:
import pickle
import numpy as np
from scipy.stats import gaussian_kde

# Paths to the pickled lists
paths = [
    "C:\\Users\\91931\\~\\diss\\training_combinations_1.pkl",
    "C:\\Users\\91931\\~\\diss\\training_combinations_2.pkl",
    "C:\\Users\\91931\\~\\diss\\training_combinations_3.pkl"
]

# Function to calculate density for a list of points
import numpy as np

def calculate_spread(points):
    if len(points) <= 1:
        return 0  # Return 0 if there is only one point or no points
    
    centroid = np.mean(points, axis=0)  # Compute centroid
    # Compute distances from each point to the centroid
    distances = np.linalg.norm(points - centroid, axis=1)
    # Compute standard deviation of distances
    spread = np.std(distances)
    # print("st dev calculated")
    
    return spread



In [ ]:
# Combine lists and calculate densities
combined_list = []
densities = []

for path in paths:
    with open(path, 'rb') as f:
        sublist = pickle.load(f)
        print(sublist[0])
        print(np.average([len(x) for x in sublist]))
        densities.extend([calculate_spread(plane) for plane in sublist])
        print("one pickled list done")

# Example usage:
# Print the first 10 densities
print("Densities of the first 10 lists:")
print(densities[:10])


In [ ]:
len(densities)

In [ ]:
from scipy import stats
stats.kstest(numberofsites, numberofsitesGEN)

In [ ]:
# Initialize empty lists to store the extracted values
losses = []
mses = []
rmse = []
maes = []

# Path to the file containing the output
file_path = "C:\\Users\\91931\\Downloads\\efermi test set.txt" # Replace with the actual file path

# Open the file and read lines
with open(file_path, "r") as file:
    for line in file:
        parts = line.strip().split(" - ")  # Strip whitespace and split each line
        for part in parts:
            if "loss" in part:
                losses.append(float(part.split(": ")[1]))
            elif "mean_squared_error" in part:
                mses.append(float(part.split(": ")[1]))
            elif "root_mean_squared_error" in part:
                rmse.append(float(part.split(": ")[1]))
            elif "mean_absolute_error" in part:
                maes.append(float(part.split(": ")[1]))

# Print the extracted values
print("\n Losses:", losses)
print("\n MSEs:", mses)
print("\n RMSEs:", rmse)
print("\n MAEs:", maes)


In [ ]:
import numpy as np
print(np.average(losses))
print(np.average(mses))
print(np.average(rmse))
print(np.average(maes))